In [95]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.tool import FunctionTool
import os, requests, json
import asyncio
from openai.types.responses import ResponseTextDeltaEvent

In [2]:
load_dotenv(override=True)
perplexity_api_key = os.getenv('PERPLEXITY_API_KEY')

if perplexity_api_key:
    print(f"Perplexity API Key exists and begins {perplexity_api_key[:4]}")
else:
    print("Perplexity API Key not set (and this is optional)")

model_name="gpt-5-nano"
perp_model_name="sonar"

Perplexity API Key exists and begins pplx


In [3]:
# Perplexity tool
def search_tool_impl(query: str) -> str:
    print(f"Searching {query}")
    response = requests.post(
        url="https://api.perplexity.ai/chat/completions",
        headers={
            "Authorization": f"Bearer {perplexity_api_key}",
            "Content-Type": "application/json"
        },
        json={
            "model": perp_model_name,
            "messages": [{"role": "user", "content": query}],
            "temperature": 0
        }
    )
    response.raise_for_status()
    return response.json().choices[0].message.content

search_schema = {
    "type": "object",
    "properties": {
        "query": {"type": "string"}
    },
    "required": ["query"]
}

def on_invoke_search(ctx, args):
    return search_tool_impl(args["query"])

search_tool = FunctionTool(
    name="search_tool",
    description="Search the web using Perplexity",
    params_json_schema=search_schema,
    on_invoke_tool=on_invoke_search
)



In [96]:
# Printing tool
def print_tool_impl(text_msg: str):
    print(text_msg)

print_schema = {
    "type": "object",
    "properties": {
        "text_msg": {"type": "string"}
    },
    "required": ["text_msg"]
}

async def on_invoke_print(ctx, args):
    if isinstance(args, str):
        try:
            args = json.loads(args)   # case: JSON string
        except json.JSONDecodeError:
            text = args              # case: scalar string
        else:
            text = args["text_msg"]  # decoded object
    else:
        text = args["text_msg"]      # already dict

    print_tool_impl(text)
    return "OK"

print_tool = FunctionTool(
    name="print_tool",
    description="Print the text_msg",
    params_json_schema=print_schema,
    on_invoke_tool=on_invoke_print
)

In [129]:
instructions1 = "You are an advertising executive and a rather funny but intelligent and creative one. You are supposed to provide one liner to sale a product given by user. Provide only  the one liner and nothing else. You live upto your expectations of creating a vaey creative and smart one liner for advertising the given product. You will be provided by the user of the product, country if use and purpose of use which you can utilise to make the selection"

instructions2 = "You are an advertising executive, intelligent but very professional and serious. You are supposed to provide one liner to sale a product given by user. Provide only  the one liner and nothing else. You live upto your expectations of creating a serious one liner for advertising the given product. You will be provided by the user of the product, country if use and purpose of use which you can utilise to make the selection"

instructions3 = "You are an advertising executive. You are given one liners and are supposed to choose the best one liner for the given product. Just choose the best one liner and nothing else"

common_message = "Use search_tool for generating results. Use new one-liners not the existing ones"


In [130]:
advertising_agent_1 = Agent(
    name="Advertising agent 1",
    instructions=instructions1 + common_message,
    tools=[search_tool]
)

advertising_agent_2 = Agent(
    name="Advertising agent 2",
    instructions=instructions2 + common_message,
    tools=[search_tool]
)

In [107]:
result = Runner.run_streamed(advertising_agent_2, input="Kookabura cricket bat. Please mention the name of the LLM model used")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

UserError: Error running tool search_tool: string indices must be integers, not 'str'

In [99]:
product_search_message = "Jordan shoes"
with trace("Parallel one liners"):
    results =await asyncio.gather(
        Runner.run(advertising_agent_1, product_search_message),
        Runner.run(advertising_agent_2, product_search_message)
    )

outputs = [result.final_output for result in results]
for output in outputs:
    print(output + "\n\n")

Error getting response: Error code: 400 - {'error': {'message': "The requested model 'sonar' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}. (request_id: req_54cade09fe374837b6a29b266ebfd11c)


BadRequestError: Error code: 400 - {'error': {'message': "The requested model 'sonar' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}

Error getting response: Error code: 400 - {'error': {'message': "The requested model 'sonar' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}. (request_id: req_e65cec4c9ad346819ede2626738b31f7)


In [151]:
result_picker_agent = Agent(
    instructions=instructions3,
    model = model_name,
    name="Best one liner selector"
)

In [200]:
product = "Bagheera from Jungle Book to be purchased by Mike Tyson as pet"


In [118]:

with trace("Select best one liner"):
    results = await asyncio.gather(
        Runner.run(advertising_agent_1, product),
        Runner.run(advertising_agent_2, product),
    )
    outputs = [result.final_output for result in results]
    print(outputs)

    one_liners = f"Selected one liner:" + "\n\nOne Liners:\n\n".join(outputs)
    best = await Runner.run(result_picker_agent, one_liners)

    print(f"Best one liner:\n{best.final_output}")

['Parle G: The biscuit that’s been dunked in more Indian chai than all your childhood secrets!', 'Nourishing India’s moments with every bite—Parle G, the taste of tradition.']
Best one liner:
Parle G: The biscuit that’s been dunked in more Indian chai than all your childhood secrets!


## Now to use tools
### Following tools will be created
#### 1. Perplexity search tool
#### 2. One-liner producing agents
#### 3. Final message printing tool

In [164]:
print(search_tool)

FunctionTool(name='search_tool', description='Search the web using Perplexity', params_json_schema={'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'additionalProperties': False}, on_invoke_tool=<function on_invoke_search at 0x000001F58E1B13A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)


In [201]:
# Agent tools
advertising_agent_1_tool = advertising_agent_1.as_tool(tool_name="advertising_agent_tool_1", tool_description="The creative one")

advertising_agent_2_tool = advertising_agent_2.as_tool(tool_name="advertising_agent_tool_2", tool_description="The serious one")

In [202]:
tools = [search_tool, advertising_agent_1_tool, advertising_agent_2_tool, print_tool]

In [203]:
selection_agent_instructions = """
You are a smart one-liner selector. Your goal is to find the best one-liner in the given context. Please follow the instructions carefully.

1.Generate draft one-liners: Use both the advertising agent tools to generate two different one-liner drafts. Do not proceed until both the drafts are ready

2. Review: the drafts and use your judgement to select the single best one-liner in an unbiased way. Use the context used by the advertising agent tools to make your judgement.

3. Print: You use print_tool to print the selected result. You must ALWAYS also hand off to the manager agent to format and print the result again in formatted form

Crucial rules:
- Use the advertising tools agents to create draft. Dont create your own
- Select only one draft one-liners as best one-liner and nothing else
- Use the Model from provided tools

"""

## Handoff Agent

In [204]:
format_tool_instruction = "You are a formatting and translation expert. You convert any English text to Marathi and print it"

format_agent = Agent(name="Format agent", instructions=format_tool_instruction, model=model_name)
format_tool =format_agent.as_tool(tool_name="format_tool", tool_description="Format the drafts")

handoff_tools = [format_tool]

# handoff_manager_instructions = "You are an agent to format text. Use tool format_tool to format the given text and print formatted text. Print only formatted text as final output"
handoff_manager_instructions = "You are a formatting and translation expert. You convert any English text to Marathi and print it. Use print tool to print the text"

handoff_manager = Agent(instructions=handoff_manager_instructions,name="Handsoff Agent", handoff_description="Handoff agent to be used by one-liner selector", tools=[print_tool])

In [189]:
print(handoff_manager)

Agent(name='Handsoff Agent', handoff_description='Handoff agent to be used by one-liner selector', tools=[FunctionTool(name='print_tool', description='Print the text_msg', params_json_schema={'type': 'object', 'properties': {'text_msg': {'type': 'string'}}, 'required': ['text_msg'], 'additionalProperties': False}, on_invoke_tool=<function on_invoke_print at 0x000001F5950B8F40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions='You are a formatting and translation expert. You convert any English text to Marathi and print it. Use print tool to print the text', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_

In [205]:
one_liner_selector = Agent(name="Selector", instructions=selection_agent_instructions + "\n\n\n Product context given to the advertising agent tools: " + product, tools=tools, handoffs=[handoff_manager])

In [206]:
selector_message = "Pick the best one-liner. Please mention the LLM model name used"
with trace("Selector verdict"):
    result = await Runner.run(one_liner_selector, selector_message)

Unleash primal power and elegance—Bagheera, the ultimate feline companion for the bold and legendary. (Model: advertising_agent_tool_2)
मूल आणि तेजाचा आवेश अनुभव—बघिरा, धैर्यवान आणि दंतकथेसारख्या व्यक्तींसाठी अंतिम मांजर साथीदार. (मॉडेल: advertising_agent_tool_2)
